In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable

%matplotlib inline

In [5]:
df = pd.read_csv('Featured_Engineering.csv')

In [6]:
df.head()

,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,bmi,stroke,Male,Other_gender,Residence,log_avg_glucose_level,formerly smoked,never smoked,smokes
0,3.0,0,0,1,1,95.12,18.0,0,1,0,0,4.555139,0,0,0
1,58.0,1,0,2,3,87.96,39.2,0,1,0,1,4.476882,0,1,0
2,8.0,0,0,1,3,110.89,17.6,0,0,0,1,4.708539,0,0,0
3,70.0,0,0,2,3,69.04,35.9,0,0,0,0,4.234686,1,0,0
4,14.0,0,0,1,1,161.28,19.1,0,1,0,0,5.083142,0,0,0


In [7]:
df.columns

Index(['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'avg_glucose_level', 'bmi', 'stroke', 'Male', 'Other_gender',
       'Residence', 'log_avg_glucose_level', 'formerly smoked', 'never smoked',
       'smokes'],
      dtype='object')

In [8]:
train_df = df[['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'bmi', 'stroke', 'Male', 'Other_gender',
       'Residence', 'log_avg_glucose_level', 'formerly smoked', 'never smoked',
       'smokes']]

In [9]:
train_df.head()

,age,hypertension,heart_disease,ever_married,work_type,bmi,stroke,Male,Other_gender,Residence,log_avg_glucose_level,formerly smoked,never smoked,smokes
0,3.0,0,0,1,1,18.0,0,1,0,0,4.555139,0,0,0
1,58.0,1,0,2,3,39.2,0,1,0,1,4.476882,0,1,0
2,8.0,0,0,1,3,17.6,0,0,0,1,4.708539,0,0,0
3,70.0,0,0,2,3,35.9,0,0,0,0,4.234686,1,0,0
4,14.0,0,0,1,1,19.1,0,1,0,0,5.083142,0,0,0


## Train test split

In [10]:
X = train_df.drop('stroke',axis=1).values

In [12]:
X.shape

(41938, 13)

In [13]:
y= train_df['stroke'].values

In [14]:
y.shape

(41938,)

## Random Over Sampling

In [28]:
from imblearn.over_sampling import RandomOverSampler

In [29]:
ros = RandomOverSampler(random_state = 42)

In [30]:
X_resampled , y_resampled = ros.fit_resample(X,y)

In [31]:
X_resampled.shape, y_resampled.shape

((82590, 13), (82590,))

In [32]:
X_resampled.shape

(82590, 13)

In [33]:
X.shape

(41938, 13)

## Train Test split

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
 X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=42)

## Scaling

In [60]:
from sklearn.preprocessing import MinMaxScaler

In [61]:
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

## Xgboost

In [62]:
from xgboost import  XGBClassifier
from sklearn.model_selection import GridSearchCV

In [63]:
xgb_classifier = XGBClassifier()

In [64]:
params = [{'max_depth' : [3,4,7],
          'learning_rate':[0.001,0.01,0.1]}]

In [65]:
GridSearch_XGB = GridSearchCV(xgb_classifier,params,cv=10)

In [66]:
GS_XGB = GridSearch_XGB.fit(scaled_X_train,y_train)

In [67]:
GS_XGB.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [68]:
GS_XGB.best_score_

0.9279119023742357

In [69]:
GS_XGB.best_params_

{'learning_rate': 0.1, 'max_depth': 7}

In [75]:
xgb_classifier_tuned = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [76]:
xgb_classifier_tuned.fit(scaled_X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [78]:
xgb_predictions = xgb_classifier_tuned.predict(scaled_X_test)

In [195]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score

In [81]:
print(classification_report(xgb_predictions,y_test))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92     11905
           1       0.99      0.88      0.93     15350

    accuracy                           0.93     27255
   macro avg       0.93      0.93      0.93     27255
weighted avg       0.93      0.93      0.93     27255



In [82]:
print(confusion_matrix(xgb_predictions,y_test))

[[11763   142]
 [ 1885 13465]]


In [83]:
xgb_classifier_tuned.save_model('xgbmodel.h5')

# Cat boost

In [179]:
from catboost import CatBoostClassifier

In [182]:
cat_classifier = CatBoostClassifier()

In [180]:
params_cat = [{'iterations':[100,200,300,500],
              'learning_rate':[0.001,0.1,0.01]}]

In [183]:
CV_cat= GridSearchCV(cat_classifier,params_cat,cv=10)

In [184]:
CV_cat_fitted = CV_cat.fit(scaled_X_train,y_train)

0:	learn: 0.6927528	total: 108ms	remaining: 10.7s
1:	learn: 0.6923684	total: 124ms	remaining: 6.06s
2:	learn: 0.6919894	total: 137ms	remaining: 4.42s
3:	learn: 0.6916044	total: 151ms	remaining: 3.62s
4:	learn: 0.6912220	total: 164ms	remaining: 3.12s
5:	learn: 0.6908333	total: 177ms	remaining: 2.78s
6:	learn: 0.6904501	total: 191ms	remaining: 2.54s
7:	learn: 0.6900728	total: 204ms	remaining: 2.34s
8:	learn: 0.6896943	total: 218ms	remaining: 2.2s
9:	learn: 0.6893076	total: 233ms	remaining: 2.1s
10:	learn: 0.6889194	total: 247ms	remaining: 2s
11:	learn: 0.6885278	total: 258ms	remaining: 1.89s
12:	learn: 0.6881544	total: 270ms	remaining: 1.81s
13:	learn: 0.6877774	total: 285ms	remaining: 1.75s
14:	learn: 0.6873978	total: 301ms	remaining: 1.7s
15:	learn: 0.6870213	total: 312ms	remaining: 1.64s
16:	learn: 0.6866437	total: 322ms	remaining: 1.57s
17:	learn: 0.6862577	total: 329ms	remaining: 1.5s
18:	learn: 0.6858854	total: 341ms	remaining: 1.46s
19:	learn: 0.6855040	total: 351ms	remaining: 1.4

66:	learn: 0.6685703	total: 749ms	remaining: 369ms
67:	learn: 0.6682260	total: 762ms	remaining: 359ms
68:	learn: 0.6678872	total: 776ms	remaining: 348ms
69:	learn: 0.6675419	total: 787ms	remaining: 337ms
70:	learn: 0.6671971	total: 798ms	remaining: 326ms
71:	learn: 0.6668490	total: 809ms	remaining: 315ms
72:	learn: 0.6665034	total: 818ms	remaining: 303ms
73:	learn: 0.6661701	total: 826ms	remaining: 290ms
74:	learn: 0.6658303	total: 835ms	remaining: 278ms
75:	learn: 0.6654897	total: 845ms	remaining: 267ms
76:	learn: 0.6651573	total: 859ms	remaining: 257ms
77:	learn: 0.6648217	total: 873ms	remaining: 246ms
78:	learn: 0.6644807	total: 887ms	remaining: 236ms
79:	learn: 0.6641408	total: 898ms	remaining: 225ms
80:	learn: 0.6637991	total: 913ms	remaining: 214ms
81:	learn: 0.6634713	total: 924ms	remaining: 203ms
82:	learn: 0.6631386	total: 936ms	remaining: 192ms
83:	learn: 0.6628037	total: 946ms	remaining: 180ms
84:	learn: 0.6624672	total: 953ms	remaining: 168ms
85:	learn: 0.6621367	total: 965

30:	learn: 0.6814333	total: 372ms	remaining: 828ms
31:	learn: 0.6810704	total: 384ms	remaining: 816ms
32:	learn: 0.6807039	total: 393ms	remaining: 799ms
33:	learn: 0.6803370	total: 406ms	remaining: 787ms
34:	learn: 0.6799662	total: 417ms	remaining: 774ms
35:	learn: 0.6796099	total: 424ms	remaining: 754ms
36:	learn: 0.6792460	total: 434ms	remaining: 739ms
37:	learn: 0.6788787	total: 441ms	remaining: 719ms
38:	learn: 0.6785163	total: 449ms	remaining: 703ms
39:	learn: 0.6781494	total: 457ms	remaining: 686ms
40:	learn: 0.6777862	total: 466ms	remaining: 671ms
41:	learn: 0.6774301	total: 474ms	remaining: 654ms
42:	learn: 0.6770790	total: 483ms	remaining: 640ms
43:	learn: 0.6767254	total: 491ms	remaining: 625ms
44:	learn: 0.6763644	total: 499ms	remaining: 610ms
45:	learn: 0.6760038	total: 506ms	remaining: 595ms
46:	learn: 0.6756485	total: 514ms	remaining: 580ms
47:	learn: 0.6752913	total: 522ms	remaining: 565ms
48:	learn: 0.6749460	total: 530ms	remaining: 552ms
49:	learn: 0.6745896	total: 538

0:	learn: 0.6927531	total: 21.6ms	remaining: 2.14s
1:	learn: 0.6923641	total: 34.7ms	remaining: 1.7s
2:	learn: 0.6919845	total: 46.6ms	remaining: 1.51s
3:	learn: 0.6916000	total: 60.9ms	remaining: 1.46s
4:	learn: 0.6912178	total: 77.6ms	remaining: 1.47s
5:	learn: 0.6908289	total: 94.5ms	remaining: 1.48s
6:	learn: 0.6904411	total: 107ms	remaining: 1.42s
7:	learn: 0.6900536	total: 119ms	remaining: 1.37s
8:	learn: 0.6896773	total: 128ms	remaining: 1.29s
9:	learn: 0.6892949	total: 138ms	remaining: 1.24s
10:	learn: 0.6889065	total: 146ms	remaining: 1.18s
11:	learn: 0.6885148	total: 156ms	remaining: 1.14s
12:	learn: 0.6881416	total: 164ms	remaining: 1.09s
13:	learn: 0.6877643	total: 173ms	remaining: 1.06s
14:	learn: 0.6873845	total: 182ms	remaining: 1.03s
15:	learn: 0.6870069	total: 190ms	remaining: 995ms
16:	learn: 0.6866287	total: 198ms	remaining: 966ms
17:	learn: 0.6862423	total: 206ms	remaining: 940ms
18:	learn: 0.6858693	total: 214ms	remaining: 911ms
19:	learn: 0.6854942	total: 224ms	re

76:	learn: 0.6654475	total: 734ms	remaining: 219ms
77:	learn: 0.6651127	total: 744ms	remaining: 210ms
78:	learn: 0.6647754	total: 751ms	remaining: 200ms
79:	learn: 0.6644401	total: 759ms	remaining: 190ms
80:	learn: 0.6641020	total: 772ms	remaining: 181ms
81:	learn: 0.6637766	total: 784ms	remaining: 172ms
82:	learn: 0.6634461	total: 796ms	remaining: 163ms
83:	learn: 0.6631160	total: 806ms	remaining: 154ms
84:	learn: 0.6627822	total: 815ms	remaining: 144ms
85:	learn: 0.6624543	total: 823ms	remaining: 134ms
86:	learn: 0.6621338	total: 831ms	remaining: 124ms
87:	learn: 0.6618075	total: 841ms	remaining: 115ms
88:	learn: 0.6614723	total: 855ms	remaining: 106ms
89:	learn: 0.6611462	total: 867ms	remaining: 96.3ms
90:	learn: 0.6608144	total: 882ms	remaining: 87.2ms
91:	learn: 0.6604898	total: 890ms	remaining: 77.4ms
92:	learn: 0.6601571	total: 899ms	remaining: 67.6ms
93:	learn: 0.6598322	total: 907ms	remaining: 57.9ms
94:	learn: 0.6594980	total: 915ms	remaining: 48.1ms
95:	learn: 0.6591779	tota

...

239:	learn: 0.4090182	total: 7.44s	remaining: 8.06s
240:	learn: 0.4087360	total: 7.48s	remaining: 8.04s
241:	learn: 0.4084580	total: 7.51s	remaining: 8.01s
242:	learn: 0.4082964	total: 7.55s	remaining: 7.99s
243:	learn: 0.4079486	total: 7.58s	remaining: 7.95s
244:	learn: 0.4076097	total: 7.6s	remaining: 7.91s
245:	learn: 0.4074795	total: 7.62s	remaining: 7.87s
246:	learn: 0.4072422	total: 7.65s	remaining: 7.83s
247:	learn: 0.4070376	total: 7.68s	remaining: 7.81s
248:	learn: 0.4068494	total: 7.71s	remaining: 7.77s
249:	learn: 0.4065305	total: 7.74s	remaining: 7.74s
250:	learn: 0.4063051	total: 7.77s	remaining: 7.71s
251:	learn: 0.4061500	total: 7.8s	remaining: 7.67s
252:	learn: 0.4058036	total: 7.84s	remaining: 7.65s
253:	learn: 0.4054675	total: 7.87s	remaining: 7.62s
254:	learn: 0.4052905	total: 7.9s	remaining: 7.59s
255:	learn: 0.4050128	total: 7.93s	remaining: 7.56s
256:	learn: 0.4048310	total: 7.96s	remaining: 7.53s
257:	learn: 0.4046850	total: 7.99s	remaining: 7.5s
258:	learn: 0.40

398:	learn: 0.3783982	total: 11.8s	remaining: 2.98s
399:	learn: 0.3782623	total: 11.8s	remaining: 2.94s
400:	learn: 0.3781474	total: 11.8s	remaining: 2.91s
401:	learn: 0.3778772	total: 11.8s	remaining: 2.88s
402:	learn: 0.3775987	total: 11.8s	remaining: 2.85s
403:	learn: 0.3774267	total: 11.9s	remaining: 2.82s
404:	learn: 0.3772868	total: 11.9s	remaining: 2.79s
405:	learn: 0.3769523	total: 11.9s	remaining: 2.76s
406:	learn: 0.3767984	total: 11.9s	remaining: 2.73s
407:	learn: 0.3766883	total: 12s	remaining: 2.69s
408:	learn: 0.3765932	total: 12s	remaining: 2.66s
409:	learn: 0.3763860	total: 12s	remaining: 2.63s
410:	learn: 0.3761937	total: 12s	remaining: 2.6s
411:	learn: 0.3759944	total: 12s	remaining: 2.57s
412:	learn: 0.3758209	total: 12s	remaining: 2.54s
413:	learn: 0.3756895	total: 12.1s	remaining: 2.5s
414:	learn: 0.3754753	total: 12.1s	remaining: 2.47s
415:	learn: 0.3753676	total: 12.1s	remaining: 2.44s
416:	learn: 0.3751595	total: 12.1s	remaining: 2.41s
417:	learn: 0.3750006	tota

62:	learn: 0.3489428	total: 1.93s	remaining: 13.4s
63:	learn: 0.3480783	total: 1.96s	remaining: 13.4s
64:	learn: 0.3470791	total: 1.99s	remaining: 13.3s
65:	learn: 0.3459432	total: 2.02s	remaining: 13.3s
66:	learn: 0.3448443	total: 2.06s	remaining: 13.3s
67:	learn: 0.3429206	total: 2.08s	remaining: 13.2s
68:	learn: 0.3420564	total: 2.11s	remaining: 13.2s
69:	learn: 0.3406870	total: 2.13s	remaining: 13.1s
70:	learn: 0.3393027	total: 2.16s	remaining: 13.1s
71:	learn: 0.3382362	total: 2.19s	remaining: 13s
72:	learn: 0.3374328	total: 2.22s	remaining: 13s
73:	learn: 0.3352220	total: 2.25s	remaining: 12.9s
74:	learn: 0.3342855	total: 2.27s	remaining: 12.9s
75:	learn: 0.3329161	total: 2.3s	remaining: 12.8s
76:	learn: 0.3322041	total: 2.33s	remaining: 12.8s
77:	learn: 0.3316035	total: 2.36s	remaining: 12.8s
78:	learn: 0.3297987	total: 2.39s	remaining: 12.7s
79:	learn: 0.3279602	total: 2.42s	remaining: 12.7s
80:	learn: 0.3270998	total: 2.45s	remaining: 12.7s
81:	learn: 0.3258242	total: 2.48s	re

221:	learn: 0.2214324	total: 6.79s	remaining: 8.51s
222:	learn: 0.2209606	total: 6.83s	remaining: 8.48s
223:	learn: 0.2203649	total: 6.86s	remaining: 8.46s
224:	learn: 0.2200415	total: 6.89s	remaining: 8.43s
225:	learn: 0.2196071	total: 6.92s	remaining: 8.39s
226:	learn: 0.2187404	total: 6.95s	remaining: 8.36s
227:	learn: 0.2180410	total: 6.98s	remaining: 8.33s
228:	learn: 0.2176456	total: 7.02s	remaining: 8.3s
229:	learn: 0.2171767	total: 7.05s	remaining: 8.27s
230:	learn: 0.2166353	total: 7.09s	remaining: 8.26s
231:	learn: 0.2157797	total: 7.14s	remaining: 8.25s
232:	learn: 0.2153098	total: 7.18s	remaining: 8.23s
233:	learn: 0.2148323	total: 7.22s	remaining: 8.21s
234:	learn: 0.2141660	total: 7.26s	remaining: 8.18s
235:	learn: 0.2136470	total: 7.29s	remaining: 8.16s
236:	learn: 0.2130507	total: 7.32s	remaining: 8.13s
237:	learn: 0.2126370	total: 7.35s	remaining: 8.09s
238:	learn: 0.2121912	total: 7.39s	remaining: 8.07s
239:	learn: 0.2115915	total: 7.42s	remaining: 8.04s
240:	learn: 0

381:	learn: 0.1501197	total: 11.7s	remaining: 3.6s
382:	learn: 0.1496965	total: 11.7s	remaining: 3.57s
383:	learn: 0.1491942	total: 11.7s	remaining: 3.54s
384:	learn: 0.1490508	total: 11.7s	remaining: 3.51s
385:	learn: 0.1486765	total: 11.8s	remaining: 3.48s
386:	learn: 0.1485384	total: 11.8s	remaining: 3.44s
387:	learn: 0.1478539	total: 11.8s	remaining: 3.41s
388:	learn: 0.1476430	total: 11.9s	remaining: 3.38s
389:	learn: 0.1472691	total: 11.9s	remaining: 3.35s
390:	learn: 0.1469967	total: 11.9s	remaining: 3.32s
391:	learn: 0.1465438	total: 11.9s	remaining: 3.29s
392:	learn: 0.1461776	total: 12s	remaining: 3.26s
393:	learn: 0.1458641	total: 12s	remaining: 3.23s
394:	learn: 0.1454609	total: 12s	remaining: 3.2s
395:	learn: 0.1451962	total: 12.1s	remaining: 3.17s
396:	learn: 0.1448866	total: 12.1s	remaining: 3.14s
397:	learn: 0.1446477	total: 12.1s	remaining: 3.11s
398:	learn: 0.1444464	total: 12.1s	remaining: 3.07s
399:	learn: 0.1440498	total: 12.2s	remaining: 3.04s
400:	learn: 0.143666

In [185]:
CV_cat_fitted.best_score_

0.9642541341627275

In [188]:
print(CV_cat_fitted.best_params_)

{'iterations': 500, 'learning_rate': 0.1}


In [189]:
cat_classifier_tuned = CatBoostClassifier(iterations=500,learning_rate=0.1)

In [190]:
cat_classifier_tuned.fit(scaled_X_train,y_train)

0:	learn: 0.6350801	total: 60.7ms	remaining: 30.3s
1:	learn: 0.5909891	total: 111ms	remaining: 27.7s
2:	learn: 0.5610495	total: 163ms	remaining: 27s
3:	learn: 0.5373312	total: 207ms	remaining: 25.7s
4:	learn: 0.5187044	total: 237ms	remaining: 23.5s
5:	learn: 0.5055544	total: 270ms	remaining: 22.2s
6:	learn: 0.4938504	total: 296ms	remaining: 20.8s
7:	learn: 0.4837499	total: 314ms	remaining: 19.3s
8:	learn: 0.4756866	total: 339ms	remaining: 18.5s
9:	learn: 0.4667649	total: 369ms	remaining: 18.1s
10:	learn: 0.4549736	total: 398ms	remaining: 17.7s
11:	learn: 0.4485451	total: 428ms	remaining: 17.4s
12:	learn: 0.4447219	total: 459ms	remaining: 17.2s
13:	learn: 0.4403442	total: 485ms	remaining: 16.8s
14:	learn: 0.4360906	total: 501ms	remaining: 16.2s
15:	learn: 0.4312383	total: 526ms	remaining: 15.9s
16:	learn: 0.4281935	total: 558ms	remaining: 15.9s
17:	learn: 0.4249000	total: 588ms	remaining: 15.8s
18:	learn: 0.4220350	total: 618ms	remaining: 15.7s
19:	learn: 0.4199763	total: 648ms	remainin

161:	learn: 0.2567113	total: 4.72s	remaining: 9.85s
162:	learn: 0.2562788	total: 4.75s	remaining: 9.81s
163:	learn: 0.2558489	total: 4.77s	remaining: 9.77s
164:	learn: 0.2552295	total: 4.82s	remaining: 9.78s
165:	learn: 0.2548480	total: 4.85s	remaining: 9.76s
166:	learn: 0.2540564	total: 4.89s	remaining: 9.74s
167:	learn: 0.2535410	total: 4.91s	remaining: 9.7s
168:	learn: 0.2527620	total: 4.94s	remaining: 9.67s
169:	learn: 0.2521986	total: 4.97s	remaining: 9.64s
170:	learn: 0.2515658	total: 5s	remaining: 9.62s
171:	learn: 0.2508291	total: 5.03s	remaining: 9.59s
172:	learn: 0.2501915	total: 5.06s	remaining: 9.56s
173:	learn: 0.2495634	total: 5.09s	remaining: 9.55s
174:	learn: 0.2488787	total: 5.12s	remaining: 9.5s
175:	learn: 0.2480841	total: 5.14s	remaining: 9.47s
176:	learn: 0.2477447	total: 5.18s	remaining: 9.45s
177:	learn: 0.2469913	total: 5.21s	remaining: 9.42s
178:	learn: 0.2464322	total: 5.24s	remaining: 9.39s
179:	learn: 0.2459006	total: 5.26s	remaining: 9.35s
180:	learn: 0.245

322:	learn: 0.1724443	total: 9.19s	remaining: 5.04s
323:	learn: 0.1720533	total: 9.22s	remaining: 5.01s
324:	learn: 0.1716709	total: 9.25s	remaining: 4.98s
325:	learn: 0.1712002	total: 9.28s	remaining: 4.95s
326:	learn: 0.1707147	total: 9.3s	remaining: 4.92s
327:	learn: 0.1703984	total: 9.32s	remaining: 4.89s
328:	learn: 0.1699833	total: 9.34s	remaining: 4.85s
329:	learn: 0.1696073	total: 9.36s	remaining: 4.82s
330:	learn: 0.1691072	total: 9.38s	remaining: 4.79s
331:	learn: 0.1687439	total: 9.4s	remaining: 4.76s
332:	learn: 0.1684393	total: 9.43s	remaining: 4.73s
333:	learn: 0.1681538	total: 9.48s	remaining: 4.71s
334:	learn: 0.1676582	total: 9.51s	remaining: 4.68s
335:	learn: 0.1674139	total: 9.54s	remaining: 4.66s
336:	learn: 0.1668765	total: 9.58s	remaining: 4.63s
337:	learn: 0.1666272	total: 9.61s	remaining: 4.61s
338:	learn: 0.1660238	total: 9.64s	remaining: 4.58s
339:	learn: 0.1655702	total: 9.67s	remaining: 4.55s
340:	learn: 0.1651767	total: 9.7s	remaining: 4.52s
341:	learn: 0.1

483:	learn: 0.1210238	total: 13.7s	remaining: 453ms
484:	learn: 0.1208246	total: 13.7s	remaining: 424ms
485:	learn: 0.1207042	total: 13.8s	remaining: 396ms
486:	learn: 0.1204131	total: 13.8s	remaining: 368ms
487:	learn: 0.1200456	total: 13.8s	remaining: 339ms
488:	learn: 0.1199122	total: 13.8s	remaining: 311ms
489:	learn: 0.1196355	total: 13.9s	remaining: 283ms
490:	learn: 0.1194325	total: 13.9s	remaining: 254ms
491:	learn: 0.1191581	total: 13.9s	remaining: 226ms
492:	learn: 0.1189182	total: 13.9s	remaining: 198ms
493:	learn: 0.1187531	total: 14s	remaining: 170ms
494:	learn: 0.1186174	total: 14s	remaining: 141ms
495:	learn: 0.1183359	total: 14s	remaining: 113ms
496:	learn: 0.1180698	total: 14.1s	remaining: 84.8ms
497:	learn: 0.1178255	total: 14.1s	remaining: 56.5ms
498:	learn: 0.1177009	total: 14.1s	remaining: 28.3ms
499:	learn: 0.1175621	total: 14.1s	remaining: 0us


In [191]:
catboost_predictions = cat_classifier_tuned.predict(scaled_X_test)

In [194]:
confusion_matrix(catboost_predictions,y_test)
print(classification_report(catboost_predictions,y_test))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     12627
           1       1.00      0.93      0.96     14628

    accuracy                           0.96     27255
   macro avg       0.96      0.97      0.96     27255
weighted avg       0.97      0.96      0.96     27255



In [196]:
print(roc_auc_score(y_test,catboost_predictions))

0.9625952520515826


In [ ]:
cat_classifier_tuned.predict()

# TEST SET 

In [117]:
test = pd.read_csv('test_v2akXPA.csv')

In [118]:
test.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,NaN
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,NaN
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked


In [119]:
test=test.drop('id',axis=1)

In [120]:
test.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,Female,14.0,0,0,No,children,Rural,95.16,21.2,NaN
3,Male,28.0,0,0,No,Private,Urban,94.76,23.4,NaN
4,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked


In [121]:
test.isnull().sum()

gender                  0
age                     0
hypertension            0
heart_disease           0
ever_married            0
work_type               0
Residence_type          0
avg_glucose_level       0
bmi                   591
smoking_status       5751
dtype: int64

In [122]:
## Gender

In [123]:
test['gender'].value_counts()

Female    10957
Male       7642
Other         2
Name: gender, dtype: int64

In [124]:
gender_dummies = pd.get_dummies(test['gender'],drop_first=True)
gender_dummies.columns=['Male','Other_gender']
test = pd.concat([test,gender_dummies],axis=1)

test=test.drop('gender',axis=1)

In [125]:
test.head()

,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,Male,Other_gender
0,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked,1,0
1,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked,0,0
2,14.0,0,0,No,children,Rural,95.16,21.2,NaN,0,0
3,28.0,0,0,No,Private,Urban,94.76,23.4,NaN,1,0
4,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked,0,0


In [126]:
## hyptertension

In [127]:
test['hypertension'].value_counts()

0    16868
1     1733
Name: hypertension, dtype: int64

In [128]:
## Heat disease

In [129]:
test['heart_disease'].value_counts()

0    17707
1      894
Name: heart_disease, dtype: int64

In [130]:
## 4.ever_married

In [131]:
test['ever_married'].value_counts()

Yes    11939
No      6662
Name: ever_married, dtype: int64

In [132]:
test['ever_married']=test['ever_married'].map({'Yes':int(2),'No':int(1)})

In [133]:
test.head()

,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,Male,Other_gender
0,80.0,0,0,2,Private,Urban,83.84,21.1,formerly smoked,1,0
1,74.0,0,1,2,Self-employed,Rural,179.50,26.0,formerly smoked,0,0
2,14.0,0,0,1,children,Rural,95.16,21.2,NaN,0,0
3,28.0,0,0,1,Private,Urban,94.76,23.4,NaN,1,0
4,63.0,0,0,2,Govt_job,Rural,83.57,27.6,never smoked,0,0


In [134]:
## Work type

In [135]:
test['work_type'].value_counts()

Private          10750
Self-employed     2861
children          2613
Govt_job          2302
Never_worked        75
Name: work_type, dtype: int64

In [136]:
test['work_type']=np.where(test['work_type']=='children',"Never_worked",test['work_type'])

In [137]:
test['work_type'].value_counts()

Private          10750
Self-employed     2861
Never_worked      2688
Govt_job          2302
Name: work_type, dtype: int64

In [138]:
test['work_type'].unique()

array(['Private', 'Self-employed', 'Never_worked', 'Govt_job'],
      dtype=object)

In [139]:
test['work_type']=test['work_type'].map({'Never_worked':int(1), 'Private':int(3), 'Self-employed':int(4), 'Govt_job':int(2)})

In [140]:
## 6. Residence Type

In [141]:
test['Residence_type'].value_counts()

Urban    9310
Rural    9291
Name: Residence_type, dtype: int64

In [142]:
residence_dummies = pd.get_dummies(test['Residence_type'],drop_first=True)
residence_dummies.columns=['Residence']
test=pd.concat([test,residence_dummies],axis=1)
test.drop('Residence_type',axis=1,inplace=True)

In [143]:
test.head()

,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,bmi,smoking_status,Male,Other_gender,Residence
0,80.0,0,0,2,3,83.84,21.1,formerly smoked,1,0,1
1,74.0,0,1,2,4,179.50,26.0,formerly smoked,0,0,0
2,14.0,0,0,1,1,95.16,21.2,NaN,0,0,0
3,28.0,0,0,1,3,94.76,23.4,NaN,1,0,1
4,63.0,0,0,2,2,83.57,27.6,never smoked,0,0,0


In [144]:
## Average glucose level

In [145]:
test['log_avg_glucose_level'] = np.log(test['avg_glucose_level'])

In [146]:
test['bmi'].mean()

28.545324819544625

In [147]:
test['bmi'].median()

27.7

In [148]:
test['bmi']= test['bmi'].fillna(test['bmi'].median())

In [149]:
test.isnull().sum()

age                         0
hypertension                0
heart_disease               0
ever_married                0
work_type                   0
avg_glucose_level           0
bmi                         0
smoking_status           5751
Male                        0
Other_gender                0
Residence                   0
log_avg_glucose_level       0
dtype: int64

In [150]:
## Smoking Status

In [151]:
test['smoking_status'] = np.where(test['smoking_status'].isnull()==True,"Unknown",test['smoking_status'])

In [152]:
test.isnull().sum()

age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
avg_glucose_level        0
bmi                      0
smoking_status           0
Male                     0
Other_gender             0
Residence                0
log_avg_glucose_level    0
dtype: int64

In [153]:
test['smoking_status'].value_counts()

never smoked       6833
Unknown            5751
formerly smoked    3260
smokes             2757
Name: smoking_status, dtype: int64

In [154]:
smoke_dummies = pd.get_dummies(test['smoking_status'],drop_first=True)

test = pd.concat([test,smoke_dummies],axis=1)

test=test.drop('smoking_status',axis=1)

In [155]:
test.head()

,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,bmi,Male,Other_gender,Residence,log_avg_glucose_level,formerly smoked,never smoked,smokes
0,80.0,0,0,2,3,83.84,21.1,1,0,1,4.428910,1,0,0
1,74.0,0,1,2,4,179.50,26.0,0,0,0,5.190175,1,0,0
2,14.0,0,0,1,1,95.16,21.2,0,0,0,4.555560,0,0,0
3,28.0,0,0,1,3,94.76,23.4,1,0,1,4.551347,0,0,0
4,63.0,0,0,2,2,83.57,27.6,0,0,0,4.425685,0,1,0


In [156]:
test.columns

Index(['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'avg_glucose_level', 'bmi', 'Male', 'Other_gender', 'Residence',
       'log_avg_glucose_level', 'formerly smoked', 'never smoked', 'smokes'],
      dtype='object')

In [158]:
test_model_feed = test[['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'bmi', 'Male', 'Other_gender', 'Residence',
       'log_avg_glucose_level', 'formerly smoked', 'never smoked', 'smokes']]

In [159]:
test_model_feed

,age,hypertension,heart_disease,ever_married,work_type,bmi,Male,Other_gender,Residence,log_avg_glucose_level,formerly smoked,never smoked,smokes
0,80.0,0,0,2,3,21.1,1,0,1,4.428910,1,0,0
1,74.0,0,1,2,4,26.0,0,0,0,5.190175,1,0,0
2,14.0,0,0,1,1,21.2,0,0,0,4.555560,0,0,0
3,28.0,0,0,1,3,23.4,1,0,1,4.551347,0,0,0
4,63.0,0,0,2,2,27.6,0,0,0,4.425685,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18596,20.0,0,0,1,3,18.4,1,0,0,4.309859,1,0,0
18597,61.0,0,0,2,2,31.6,1,0,0,5.354461,0,0,1
18598,79.0,0,0,2,3,29.4,0,0,0,4.834216,0,1,0
18599,55.0,0,0,2,2,33.8,1,0,0,4.240751,0,1,0


In [160]:
test_model_feed_array = test_model_feed.values

In [161]:
test_model_feed_array

array([[80.,  0.,  0., ...,  1.,  0.,  0.],
       [74.,  0.,  1., ...,  1.,  0.,  0.],
       [14.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [79.,  0.,  0., ...,  0.,  1.,  0.],
       [55.,  0.,  0., ...,  0.,  1.,  0.],
       [38.,  0.,  0., ...,  0.,  0.,  0.]])

In [162]:
scaled_test_model_feed_array = scaler.transform(test_model_feed_array)

In [163]:
solution_ = xgb_classifier_tuned.predict(scaled_test_model_feed_array)

In [164]:
solution_

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

In [166]:
solution_df = pd.DataFrame(solution_)

In [167]:
solution_df.to_csv('Solutions.csv')

In [197]:
solution_cat = cat_classifier_tuned.predict(scaled_test_model_feed_array)

In [198]:
solution_cat_df = pd.DataFrame(solution_cat)

In [199]:
solution_cat_df.to_csv('catboost.csv')